In [3]:
pip install -U sagemaker -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'tiiuae/falcon-7b-instruct',
	'SM_NUM_GPUS': json.dumps(1)
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.0.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=1000,
  )
  
# send request
predictor.predict({
	"inputs": "My name is Julien and I like to",
})

In [ ]:
output=predictor.predict({
	"inputs": """
Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know".

Context:
Neeraj Chopra, the Indian athlete, achieved a historic milestone at the 2020 Summer Olympics held in Tokyo by securing the gold medal in the men's javelin throw event. His performance was remarkable, as he set a new national record with a throw of 87.58 meters during his second attempt. This victory was not only significant for him personally but also for India as a nation. Neeraj Chopra became the first Indian athlete to win a gold medal in track and field at the Olympic Games, a momentous achievement celebrated widely across the country. In a highly competitive field of javelin throwers from around the world, Neeraj's consistent and exceptional performance led to his well-deserved gold medal. His win had a profound impact, inspiring countless aspiring athletes in India and drawing attention to the sport of javelin throw, cementing Neeraj Chopra's status as a symbol of excellence in Indian athletics.


Ques: Who won the 2020 Summer Olympics men's Javelin throw?

Ans:

""",
})

print(output[0]['generated_text'])